In [1]:
from bs4 import BeautifulSoup 
import requests
import numpy as np
import pandas as pd
import math

# THIS ASSIGNMENT IS FOR QUESTION 1

### STEP 1: Build the code to scrape the following Wikipedia page to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [2]:
URL='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response=requests.get(URL)
html_soup=BeautifulSoup(response.text, 'html.parser')

response.status_code


200

In [3]:
postal_code = html_soup.find('table', class_ = "wikitable sortable")

In [4]:
len(postal_code)

2

### STEP 2: Manipulate the html parser and get all the necessary information in the format we need

In [5]:
Postcode= []
Borough= []
Neighbourhood = []

for tr in postal_code.find_all('tr'):

    for idx, td in enumerate(tr.find_all('td')):                
        #Postcode.append(td[0].text)
        if idx == 0:
            if td.text =="Not assigned":
                Postcode.append(np.NaN)
            else:
                Postcode.append(td.text)
            #print(td.text)
        elif idx== 1:
            if td.text =="Not assigned":
                Borough.append(np.NaN)
            else:
                Borough.append(td.text)
            #print(td.text)
        elif idx == 2:
            #if td.text.strip() =="Not assigned":
            #    Neighbourhood.append(np.NaN)
            #else:
            Neighbourhood.append(td.text.strip())
            #print(td.text)
        #print(idx)


### STEP 3: Ensure the final output from the web scraping is in pandas dataframe

In [6]:
post=pd.DataFrame({
    "Postcode" : Postcode,
    "Borough": Borough,
    "Neighbourhood": Neighbourhood
                  }) 
post.head(n=5)

,Borough,Neighbourhood,Postcode
0,NaN,Not assigned,M1A
1,NaN,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A


In [7]:
len(post)

289

### STEP 4 - Ensure all the conditions are met before finalizing the output


In [7]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
post = post.loc[post["Borough"].dropna(axis=0, how='any').index.values,]
len(post)

212

In [8]:
post.sort_values(by=['Postcode', 'Borough'],inplace=True)

In [9]:
post[post["Neighbourhood"] == "Not assigned"]

,Borough,Neighbourhood,Postcode
8,Queen's Park,Not assigned,M7A


In [10]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
# So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
post.loc[post["Neighbourhood"] == "Not assigned", ["Neighbourhood"]] = post[post["Neighbourhood"] == "Not assigned"]["Borough"]

In [11]:
post[post["Neighbourhood"] == "Not assigned"]

,Borough,Neighbourhood,Postcode


In [12]:
post[post["Postcode"] == "M7A" ] 

,Borough,Neighbourhood,Postcode
8,Queen's Park,Queen's Park,M7A


In [13]:
post.head()

,Borough,Neighbourhood,Postcode
11,Scarborough,Rouge,M1B
12,Scarborough,Malvern,M1B
27,Scarborough,Highland Creek,M1C
28,Scarborough,Rouge Hill,M1C
29,Scarborough,Port Union,M1C


In [14]:
# More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
#These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

post = pd.DataFrame(post.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x)), index=None)
post.reset_index(inplace = True)                    

In [15]:
post.head(n=5)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [16]:
# last cell of your notebook, use the .shape method to print the number of rows of your dataframe
post.shape

(103, 3)

## THIS IS THE END OF ASSIGNMENT FOR QUESTION 1